# Aufgabe 3

### Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 500)

# a)

### Read data

In [ ]:
employment = pd.read_excel("dataset/employment_08_09.xlsx")
print(employment.shape)

In [ ]:
employment.head()

### Variable overview

The data set contains mainly discrete variables:
    
- The variable `race` is a categorical nominal-scaled variable wich encodes three distinct states.
- The variable `age` is an integer ratio-scaled variable.
- The rest (exept `earnwke`) are binary (=dummy) variables.

The variable `earnwke` is a continuous variable that is ratio-scaled.

1. The target variable is `employed` / `unemployed`. They should be distinct and exhaustive. Check and leave one out for later analysis.

2. The variable `race` encodes three possible states. Transform to dummies and leave one out for later analysis.

3. The variables `ne_states`, `so_states`, `ce_states` and `we_states` should be distinct and exhaustive. Check and leave one out for later analysis.

4. The variables `private`, `government` and `self` should be distinct and exhaustive. Check and leave one out for later analysis.

5. The variables `educ_lths`, `educ_hs`, `educ_somecol`, `educ_aa`, `educ_ba` and `educ_adv` should be distinct and exhaustive. Check and leave one out for later analysis.

6. The variable `earnwke` is probably not available for self-employed people. Check later and handle possible missing values.

### 1. Sanity check target variables

In [ ]:
employment[["employed", "unemployed"]].value_counts()

One can see, that there are 435 data points which contain no information about wether or not the person was employed in 2009.

These points will be removed from the data.

In [ ]:
unknown_employment_status = (employment["employed"] == employment["unemployed"])
employment = employment[~unknown_employment_status]
employment.head()

Now we remove the redundant variable `employed`:

In [ ]:
employment.drop(columns="employed", inplace=True)
employment.head()

### 2. Dummy transform `race` column

In [ ]:
employment = pd.get_dummies(employment, columns=["race"], drop_first=True)
employment.head()

### 3. Remove redundancy in terms of employment states

Check if every worker contains one and only one employment state:

In [ ]:
employment[["ne_states", "so_states", "ce_states", "we_states"]].sum(axis=1).value_counts()

Drop first state indicator to remove redundancy:

In [ ]:
employment.drop(columns="ne_states", inplace=True)
employment.head()

### 4. Remove redundancy in type of employment

Check if every worker contains one and only one employment type:

In [ ]:
employment[["government", "private", "self"]].sum(axis=1).value_counts()

Drop `self` employement indicator to remove redundancy and to reduce maximal amout of correlation between variables (we know when union=1 => self=0):

In [ ]:
employment.drop(columns="self", inplace=True)
employment.head()

### 5. Remove redundancy in worker education

Check if every worker contains one and only one type of highest education level:

In [ ]:
employment[["educ_lths", "educ_hs", "educ_somecol", "educ_aa", "educ_bac", "educ_adv"]].sum(axis=1).value_counts()

Drop first education level to remove redundancy:

In [ ]:
employment.drop(columns="educ_lths", inplace=True)
employment.head()

### 6. Drop `earnwke` variable due to missing values for self-employed workers

In [ ]:
employment.drop(columns="earnwke", inplace=True)
employment.head()

### Check distribution of target variable

In [ ]:
target_group = employment["unemployed"].value_counts()
print("Amount of unemployment vs. employment in data: {n_unemployed} vs. {n_employed}".format(n_employed=target_group[0], n_unemployed=target_group[1]))

One can see that unemployment is pretty undersampled in the training data. One could oversample it by bootstrapping or undersample employment by leaving out random data points.

### Define input and target variables

In [ ]:
X = employment.drop(columns="unemployed")
y = employment["unemployed"]

### Get train/test split

In [ ]:
input_train, input_test, label_train, label_test = train_test_split(X, y, train_size=0.8, shuffle=True, stratify=y)

### Possible methods

We want to predict who is going to be unemployed in the future. The target variable is discrete (more specifically binary) and that's why we are looking at a classification task with two classes.

Possible classification algorithms are:
- k-Nearest-Neighbour
- Decision Tree
- Logistic Regression
- Support Vector Machine

Choosing Logistic Regression, because it is very interesting to compare the L2 loss against the cross entropy loss in terms of classification.

### Baseline: Most often class

In [ ]:
most_often_class = label_train.mode()[0]
print(f"The class that appears most often in the training data is: class {most_often_class} (= employed)")

In [ ]:
model = lambda x: most_often_class
y_hat = input_test.apply(model, axis=1)
print(f"The accuracy of this model is: {accuracy_score(label_test, y_hat):.2%}")

### Baseline: Linear Regression

In [ ]:
model = linear_model.LinearRegression()
model.fit(input_train, label_train)

In [ ]:
plt.rcParams["figure.figsize"] = (11, 9)

y_hat = model.predict(input_test)

print(f"accuracy: {accuracy_score(label_test, y_hat.round()):.2%}")

cm = confusion_matrix(label_test, y_hat.round())
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()

### Logistic Regression

In [ ]:
model = linear_model.LogisticRegression(solver="liblinear").fit(input_train, label_train)

In [ ]:
cm = confusion_matrix(label_test, y_hat.round())
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()

print(f"The mean accuracy is: {model.score(input_test, label_test):.2%}")

### Comparison

All methods reached the same accuracy score. This is the case, because every sample in the test set was classified as `employed`. It is possible that this happened, because the class `unemployed` is undersampled in the data.

### Were older employeed more probable to be unemployed after the financial crisis in 2008/2009?

In [ ]:
fig = px.histogram(employment, x="age", y="unemployed", nbins=5)
fig.show()

It seems that for this data, the age played no role in somebody losing his job in 2009.

# b)

### Read data

In [3]:
growth = pd.read_excel("dataset/Growth.xlsx")
growth.head()

/Users/lukaskubelka/Documents/_KIT/_Studium/_M.Sc./_Semester/Semester-2/PSDA/Uebungen/E1/.venv/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,country_name,growth,oil,rgdp60,tradeshare,yearsschool,rev_coups,assasinations
0,India,1.915168,0,765.999817,0.140502,1.45,0.133333,0.866667
1,Argentina,0.617645,0,4462.001465,0.156623,4.99,0.933333,1.933333
2,Japan,4.304759,0,2953.999512,0.157703,6.71,0.000000,0.200000
3,Brazil,2.930097,0,1783.999878,0.160405,2.89,0.100000,0.100000
4,United States,1.712265,0,9895.003906,0.160815,8.66,0.000000,0.433333


The data set contains mainly continuous variables:
- the variables `growth`, `rgdp60`, `tradeshare`, `yearsschool`, `rev_coups` and `assassinations` are all real-valued

The variables `country_name` and `oil` are discrete:
- `county_name` is a categorical variable (irrelevant)
- `oil` is a binary variable

The target variable is `growth`

That means we have a regression task

### Get familiar with data

In [4]:
growth.sort_values(by="assasinations", ascending=False).head()

,country_name,growth,oil,rgdp60,tradeshare,yearsschool,rev_coups,assasinations
28,Guatemala,0.929231,0,1659.999756,0.509590,1.42,0.540741,2.466667
1,Argentina,0.617645,0,4462.001465,0.156623,4.99,0.933333,1.933333
30,El Salvador,-0.607556,0,1426.999878,0.527773,1.70,0.633333,1.733333
6,Spain,2.880327,0,3123.001465,0.299406,3.80,0.066667,1.433333
11,Italy,2.932981,0,4564.000488,0.330022,4.56,0.033333,1.200000


In [5]:
growth.sort_values(by="yearsschool", ascending=False).head()

,country_name,growth,oil,rgdp60,tradeshare,yearsschool,rev_coups,assasinations
37,Denmark,2.179370,0,6759.999023,0.560750,10.07,0.0,0.000000
23,New Zealand,1.124132,0,7960.000977,0.455769,9.55,0.0,0.000000
10,Australia,1.975147,0,7782.002441,0.329479,9.03,0.0,0.066667
4,United States,1.712265,0,9895.003906,0.160815,8.66,0.0,0.433333
21,Canada,2.385988,0,7257.997070,0.444375,8.07,0.0,0.066667


In [6]:
growth[growth["country_name"] == "Germany"]

,country_name,growth,oil,rgdp60,tradeshare,yearsschool,rev_coups,assasinations
20,Germany,2.453688,0,6569.998535,0.442143,7.64,0.066667,0.233333


### 1. Check if variable `oil` contains any information

In [7]:
growth["oil"].value_counts()

0    65
Name: oil, dtype: int64

All values of the variable `oil` are zero. That means it has to be dropped:

In [8]:
growth.drop(columns="oil", inplace=True)
growth.head()

,country_name,growth,rgdp60,tradeshare,yearsschool,rev_coups,assasinations
0,India,1.915168,765.999817,0.140502,1.45,0.133333,0.866667
1,Argentina,0.617645,4462.001465,0.156623,4.99,0.933333,1.933333
2,Japan,4.304759,2953.999512,0.157703,6.71,0.000000,0.200000
3,Brazil,2.930097,1783.999878,0.160405,2.89,0.100000,0.100000
4,United States,1.712265,9895.003906,0.160815,8.66,0.000000,0.433333


### 2. Drop variable `country_name`, because it is not a good predictor

In [9]:
growth.drop(columns="country_name", inplace=True)
growth.head()

,growth,rgdp60,tradeshare,yearsschool,rev_coups,assasinations
0,1.915168,765.999817,0.140502,1.45,0.133333,0.866667
1,0.617645,4462.001465,0.156623,4.99,0.933333,1.933333
2,4.304759,2953.999512,0.157703,6.71,0.000000,0.200000
3,2.930097,1783.999878,0.160405,2.89,0.100000,0.100000
4,1.712265,9895.003906,0.160815,8.66,0.000000,0.433333


### Choose target variable and regressors

In [40]:
y = growth["growth"]
X = growth.drop(columns="growth")

In [41]:
fig = px.histogram(growth, x=y, nbins=10)
fig.show()

### Possible methods

Possible regression methods are:

- ridge regression
- regression tree
- knn regression

Ridge Regression will be used, because the target variable could be assumed to be normal distributed and it is generally a good idea to use a shrinkidge model to prevent overfitting (likely to happen with this many training examples).

### Create training and test-split

In [47]:
input_train, input_test, target_train, target_test = train_test_split(X, y, shuffle=True)

### Baseline

In [49]:
avg_growth = target_train.mean()
model = lambda x: avg_growth
y_hat = input_test.apply(model, axis=1)
print(f"MSE of baseline is: {mean_squared_error(target_test, y_hat)}")

MSE of baseline is: 3.2778506858920724


### Ridge Regression

Preprocessing:

For ridge regression one has to standardize the features by removing the mean and scaling to unit variance.

In [50]:
input_train = StandardScaler().fit_transform(input_train)
input_test = StandardScaler().fit_transform(input_test)

In [51]:
model = linear_model.Ridge(alpha=0.1)
model.fit(input_train, target_train)

Ridge(alpha=0.1)

In [52]:
y_hat = model.predict(input_test)

print("MSE: ", mean_squared_error(target_test, y_hat))
print("R2: ", r2_score(target_test, y_hat))

MSE:  1.601046695947192
R2:  0.5096049615792992


Evaluation: ...

# c)

### Read data

In [ ]:
crabs = pd.read_csv("dataset/crabs.txt", header=None, delim_whitespace=True, names=["id", "color", "spine", "width", "weight", "satellites"], index_col=0)

In [ ]:
crabs.head()

### Get familiar with data

In [ ]:
crabs.sort_values(by="satellites", ascending=False)

Target variable is `satellites`.

### Transform data

In [ ]:
crabs = pd.get_dummies(crabs, columns=["color", "spine"], drop_first=True)
crabs.head()

In [ ]:
fig = px.histogram(crabs["satellites"])
fig.show()

# d)

In [ ]:
gasoline = pd.read_csv("dataset/gasoline.csv", sep=";", index_col=0)

In [ ]:
gasoline.head()

# Präsentation

Was ist Lesson learned? Was war interessant, was war wichtig bei der Aufgabe?